# RECMORISATION 

Conda environment created with cmor v3.12.0 a recent version of iris and ipykernel to allow interface with Jupyterhub

In [1]:
import cmor  # used for writing files
import iris  # used for reading files -- netCDF4 or xarray could be used here based on preference
import json
import os

### CMOR input json

In [2]:
DATASET_INFO = {
    "_AXIS_ENTRY_FILE": "tables/CMIP7_coordinate.json",
    "_FORMULA_VAR_FILE": "tables/CMIP7_formula_terms.json",
    "_cmip7_option": 1,
    "_controlled_vocabulary_file": "test/CMIP7-CV_for-cmor.json",
    "activity_id": "CMIP",
    "branch_method": "standard",
    "branch_time_in_child": 30.0,
    "branch_time_in_parent": 10800.0,
    "calendar": "360_day",
    "cv_version": "7.0.0.0",
    "drs_specs": "MIP-DRS7",
    "experiment": "1 percent per year increase in CO2",
    "experiment_id": "1pctCO2",
    "forcing_index": "f3",
    "grid": "N96",
    "grid_label": "gn",
    "initialization_index": "i1",
    "institution_id": "PCMDI",
    "license_id": "CC BY 4.0",
    "nominal_resolution": "250 km",
    "outpath": ".",
    "parent_mip_era": "CMIP7",
    "parent_time_units": "days since 1850-01-01",
    "parent_activity_id": "CMIP",
    "parent_source_id": "PCMDI-test-1-0",
    "parent_experiment_id": "piControl",
    "parent_variant_label": "r1i1p1f3",
    "physics_index": "p1",
    "realization_index": "r1",
    "source_id": "PCMDI-test-1-0",
    "source_type": "AOGCM CHEM BGC",
    "tracking_prefix": "hdl:21.14100",
    "host_collection": "CMIP7",
    "frequency": "mon",
    "region": "glb",
    "archive_id": "WCRP",
    "mip_era": "CMIP7",
}

with open('input.json', 'w') as fh:
    json.dump(DATASET_INFO, fh, indent=2)

### Get hold of data

In [3]:
LOCAL_CMIP6_ROOT = '/badc/cmip6/data/'  # will be different in other places


# Load all existing monthly tas data as a single iris cube;

cubelist = iris.load(LOCAL_CMIP6_ROOT + 'CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r1i1p1f2/Amon/tas/gn/v20190406/tas*.nc')

for i in cubelist:
    i.attributes = {}

tas_cube = cubelist.concatenate_cube()

# load all existing monthly pr data as a single iris cube

cubelist = iris.load(LOCAL_CMIP6_ROOT + 'CMIP6/CMIP/MOHC/UKESM1-0-LL/1pctCO2/r1i1p1f2/Amon/pr/gn/v20190406/pr*.nc')

for i in cubelist:
    i.attributes = {}

pr_cube = cubelist.concatenate_cube()

# warnings related to missing CF netcdf or pyproj can be ignored.

/home/users/msmizielinski/.conda/envs/CMOR-3.12/lib/python3.13/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/home/users/msmizielinski/.conda/envs/CMOR-3.12/lib/python3.13/site-packages/iris/fileformats/cf.py:934: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/users/msmizielinski/.conda/envs/CMOR-3.12/lib/python3.13/site-packages/iris/fileformats/cf.py:934: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'pr'
  warnings.warn(


### Setup CMOR

In [4]:
cmor.setup(inpath="development-tables", netcdf_file_action=cmor.CMOR_REPLACE)

cmor.dataset_json('input.json')

0

In [5]:
cmor.load_table('CMIP7_atmos.json')

# warnings in this can be ignored -- the metadata in the tables is not perfectly set up


C Traceback:
In function: cmor_load_table_internal
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: unknown section: version_metadata, for table: 
!
!!!!!!!!!!!!!!!!!!!!!!!!!


C Traceback:
In function: cmor_load_table_internal
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: attribute for unknown section: version_metadata,{ "version_tag": "v6.5.0.0", "checksum": "md5: c4c59dea7b58b910a6dc5345e31638f6", "Auxillary_files\/MIP_formula_terms_modified": "Tue Nov 28 21:14:07 2023 +0000", "Auxillary_files\/MIP_formula_terms_note": "Updating versioning and model_components scripts to process files individually.", "commit": "50ee06a3b58fb2b064485d454203766327f4d770", "previous_commit": "", "author": "Daniel Ellis", "CV_collection_version": "v6.5.1.1", "specs_doc": "v6.5.0" } (table: )
!
!!!!!!!!!!!!!!!!!!!!!!!!!



0

### Write tas (50 record chunks)


In [6]:
# construct latitude and longitude coords
cmorlat = cmor.axis("latitude",
                    coord_vals=tas_cube.coord('latitude').points,
                    cell_bounds=tas_cube.coord('latitude').bounds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=tas_cube.coord('longitude').points,
                    cell_bounds=tas_cube.coord('longitude').bounds,
                    units="degrees_east")
# construct time coord in a way that we can update with points and bounds later
cmortime = cmor.axis("time", units="days since 1850-01-01")
# define CMOR variable object
axes = [cmortime, cmorlat, cmorlon]
cmortas = cmor.variable("tas_tavg-h2m-hxy-u", "K", axes)

# slice up data into N time record chunks and push through CMOR.write
N = 50
for i in range(0, 1800, N):
    s = slice(i, i+N)
    print(s)
    cube_slice = tas_cube[s]
    cmor.write(
        cmortas,
        cube_slice.data, 
        time_vals=cube_slice.coord('time').points, 
        time_bnds=cube_slice.coord('time').bounds)

# close the file (sorts the full naming)
cmor.close(cmortas, file_name=True)

slice(0, 50, None)
slice(50, 100, None)
slice(100, 150, None)
slice(150, 200, None)
slice(200, 250, None)
slice(250, 300, None)
slice(300, 350, None)
slice(350, 400, None)
slice(400, 450, None)
slice(450, 500, None)
slice(500, 550, None)
slice(550, 600, None)
slice(600, 650, None)
slice(650, 700, None)
slice(700, 750, None)
slice(750, 800, None)
slice(800, 850, None)
slice(850, 900, None)
slice(900, 950, None)
slice(950, 1000, None)
slice(1000, 1050, None)
slice(1050, 1100, None)
slice(1100, 1150, None)
slice(1150, 1200, None)
slice(1200, 1250, None)
slice(1250, 1300, None)
slice(1300, 1350, None)
slice(1350, 1400, None)
slice(1400, 1450, None)
slice(1450, 1500, None)
slice(1500, 1550, None)
slice(1550, 1600, None)
slice(1600, 1650, None)
slice(1650, 1700, None)
slice(1700, 1750, None)
slice(1750, 1800, None)


'./CMIP7/CMIP/PCMDI-test-1-0/1pctCO2/r1i1p1f3/atmos/mon/tas/tavg-h2m-hxy-u/gn/v20250910/tas_tavg-h2m-hxy-u_mon_glb_gn_PCMDI-test-1-0_1pctCO2_r1i1p1f3_185001-199912.nc'

### Write pr (50 record chunks)

In [7]:
# construct latitude and longitude coords
cmorlat = cmor.axis("latitude",
                    coord_vals=pr_cube.coord('latitude').points,
                    cell_bounds=pr_cube.coord('latitude').bounds,
                    units="degrees_north")
cmorlon = cmor.axis("longitude",
                    coord_vals=pr_cube.coord('longitude').points,
                    cell_bounds=pr_cube.coord('longitude').bounds,
                    units="degrees_east")
# construct time coord in a way that we can update with points and bounds later
cmortime = cmor.axis("time",
                        units="days since 1850-01-01")
# define CMOR variable object
axes = [cmortime, cmorlat, cmorlon]
cmorpr = cmor.variable("pr_tavg-u-hxy-u", "kg m-2 s-1", axes)

# slice up data into N time record chunks and push through CMOR.write
N = 50
for i in range(0, 1800, N):
    s = slice(i, i + N)
    print(s)
    cube_slice = pr_cube[s]
    cmor.write(
        cmorpr,
        tas_cube[s].data, 
        time_vals=tas_cube.coord('time').points[s], 
        time_bnds=tas_cube.coord('time').bounds[s])

# close the file (sorts the full naming)
cmor.close(cmorpr, file_name=True)

slice(0, 50, None)
slice(50, 100, None)
slice(100, 150, None)
slice(150, 200, None)
slice(200, 250, None)
slice(250, 300, None)
slice(300, 350, None)
slice(350, 400, None)
slice(400, 450, None)
slice(450, 500, None)
slice(500, 550, None)
slice(550, 600, None)
slice(600, 650, None)
slice(650, 700, None)
slice(700, 750, None)
slice(750, 800, None)
slice(800, 850, None)
slice(850, 900, None)
slice(900, 950, None)
slice(950, 1000, None)
slice(1000, 1050, None)
slice(1050, 1100, None)
slice(1100, 1150, None)
slice(1150, 1200, None)
slice(1200, 1250, None)
slice(1250, 1300, None)
slice(1300, 1350, None)
slice(1350, 1400, None)
slice(1400, 1450, None)
slice(1450, 1500, None)
slice(1500, 1550, None)
slice(1550, 1600, None)
slice(1600, 1650, None)
slice(1650, 1700, None)
slice(1700, 1750, None)
slice(1750, 1800, None)


'./CMIP7/CMIP/PCMDI-test-1-0/1pctCO2/r1i1p1f3/atmos/mon/pr/tavg-u-hxy-u/gn/v20250910/pr_tavg-u-hxy-u_mon_glb_gn_PCMDI-test-1-0_1pctCO2_r1i1p1f3_185001-199912.nc'